# Without Feature Engineering with isDebt Column

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import pickle

In [ ]:
df=pd.read_csv('./Dataset/bank_cleansed.csv')
df

,age,default,balance,housing,loan,campaign,pdays,previous,deposit,month_num,...,categorical__month_jun,categorical__month_mar,categorical__month_may,categorical__month_nov,categorical__month_oct,categorical__month_sep,categorical__poutcome_failure,categorical__poutcome_other,categorical__poutcome_success,categorical__poutcome_unknown
0,59,0,2343,1,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,56,0,45,0,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,41,0,1270,1,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,55,0,2476,1,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,54,0,184,0,0,2,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,0,1,1,0,1,-1,0,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11158,39,0,733,0,0,4,-1,0,0,6,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11159,32,0,29,0,0,2,-1,0,0,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11160,43,0,0,0,1,2,172,5,0,5,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
seed=67
X=df.drop(columns='deposit')
y=df['deposit']
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=seed,test_size=0.2)

In [ ]:
xgb=XGBClassifier()
params_grid={
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1]
}
gs=GridSearchCV(estimator=xgb,param_grid=params_grid,scoring='roc_auc')
gs.fit(X_train,y_train)
best_model_without_feature_en=gs.best_estimator_

In [ ]:
with open("./Models/model_without_feature_en.pkl", "wb") as f:
    pickle.dump(best_model_without_feature_en, f)

In [13]:
with open("Models/model_without_feature_en.pkl", "rb") as f:
    model = pickle.load(f)
pred=model.predict(X_test)
y_proba=model.predict_proba(X_test)[:,1]
print(confusion_matrix(y_true=y_test,y_pred=pred))
print(classification_report(y_true=y_test,y_pred=pred))
print(roc_auc_score(y_test,y_proba))

[[985 190]
 [429 629]]
              precision    recall  f1-score   support

           0       0.70      0.84      0.76      1175
           1       0.77      0.59      0.67      1058

    accuracy                           0.72      2233
   macro avg       0.73      0.72      0.72      2233
weighted avg       0.73      0.72      0.72      2233

0.7729236214455215


In [16]:
feature_importance=pd.DataFrame(data={'feature_names':model.feature_names_in_,'feature_importance':model.feature_importances_}).sort_values(by='feature_importance',ascending=False)
feature_importance

,feature_names,feature_importance
45,categorical__poutcome_success,0.204153
30,categorical__contact_unknown,0.167626
3,housing,0.045817
38,categorical__month_mar,0.042671
41,categorical__month_oct,0.034084
42,categorical__month_sep,0.032219
37,categorical__month_jun,0.029930
46,categorical__poutcome_unknown,0.029671
31,categorical__month_apr,0.026319
28,categorical__contact_cellular,0.021792


In [21]:
df2=pd.read_csv('./Dataset/bank_featured.csv')
df2

,age,default,balance,housing,loan,campaign,pdays,previous,deposit,month_num,...,categorical__poutcome_success,categorical__poutcome_unknown,categorical__balance_bin_-0,categorical__balance_bin_0-500,categorical__balance_bin_10001+,categorical__balance_bin_1001-2000,categorical__balance_bin_2001-5000,categorical__balance_bin_5001-10000,categorical__balance_bin_501-1000,is_debt
0,59,0,2343,1,0,1,-1,0,1,5,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,56,0,45,0,0,1,-1,0,1,5,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,41,0,1270,1,0,1,-1,0,1,5,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,55,0,2476,1,0,1,-1,0,1,5,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,54,0,184,0,0,2,-1,0,1,5,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,0,1,1,0,1,-1,0,0,4,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
11158,39,0,733,0,0,4,-1,0,0,6,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
11159,32,0,29,0,0,2,-1,0,0,8,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
11160,43,0,0,0,1,2,172,5,0,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [22]:
seed=67
X2=df2.drop(columns='deposit')
y2=df2['deposit']
X2_train,X2_test,y2_train,y2_test=train_test_split(X2,y2,stratify=y2,random_state=seed,test_size=0.2)

In [24]:
xgb=XGBClassifier()
params_grid = param_grid = {
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1]
}
gs2=GridSearchCV(estimator=xgb,param_grid=params_grid,scoring='roc_auc')
gs2.fit(X2_train,y2_train)
best_model_with_feature_en=gs2.best_estimator_

In [25]:
with open("./Models/model_with_feature_en.pkl", "wb") as f:
    pickle.dump(best_model_with_feature_en, f)

In [27]:
with open("Models/model_with_feature_en.pkl", "rb") as f:
    model2 = pickle.load(f)
pred2=model2.predict(X2_test)
y2_proba=model2.predict_proba(X2_test)[:,1]
print(confusion_matrix(y_true=y2_test,y_pred=pred2))
print(classification_report(y_true=y2_test,y_pred=pred2))
print(roc_auc_score(y2_test,y2_proba))

[[985 190]
 [425 633]]
              precision    recall  f1-score   support

           0       0.70      0.84      0.76      1175
           1       0.77      0.60      0.67      1058

    accuracy                           0.72      2233
   macro avg       0.73      0.72      0.72      2233
weighted avg       0.73      0.72      0.72      2233

0.7717640670876403


In [28]:
feature_importance2=pd.DataFrame(data={'feature_names':model2.feature_names_in_,'feature_importance':model2.feature_importances_}).sort_values(by='feature_importance',ascending=False)
feature_importance2

,feature_names,feature_importance
45,categorical__poutcome_success,0.173280
46,categorical__poutcome_unknown,0.140246
30,categorical__contact_unknown,0.129512
3,housing,0.037159
38,categorical__month_mar,0.034812
41,categorical__month_oct,0.033956
28,categorical__contact_cellular,0.025643
37,categorical__month_jun,0.022200
32,categorical__month_aug,0.021961
31,categorical__month_apr,0.021594
